# Human Action Recognition with Optical-Flow, Bag-Of-Words and SVM

## 0. Import Libraries

In [1]:
import cv2
import numpy as np
import os
import pickle
from sklearn.cluster import KMeans
from numpy import size
from scipy.cluster.vq import vq
from sklearn.svm import SVC

## 1. Extract Optical Flow
Extract optical flow in the x and y direction for each frame in all directions

In [2]:
CATEGORIES = ["boxing", "handclapping", "handwaving", "jogging", "running", "walking"]

# Create directory to store extracted SIFT features.
os.makedirs("data", exist_ok=True)

# Setup parameters for optical flow.
farneback_params=dict(winsize=20,iterations=1,flags=cv2.OPTFLOW_FARNEBACK_GAUSSIAN,levels=1,pyr_scale=0.5,poly_n=5,poly_sigma=1.1,flow=None)

n_processed_files = 0

for category in CATEGORIES:
    print("Processing category %s" % category)

    # Get all files in current category's folder.
    folder_path = os.path.join("..", "dataset", category)
    filenames = os.listdir(folder_path)

    # List to store features. features[i] stores features for the i-th video
    # in current category.
    features = []

    for filename in filenames:
        filepath = os.path.join("..", "dataset", category, filename)
        vid = cv2.VideoCapture(filepath)

        # Store features in current file.
        features_current_file = []

        prev_frame = None

        while vid.isOpened():
            ret, frame = vid.read()
            if not ret:
                break

            # Only care about gray scale.
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            if prev_frame is not None:
                # Calculate optical flow.
                flows = cv2.calcOpticalFlowFarneback(prev_frame, frame, **farneback_params)

                feature = []
                for r in range(120):
                    if r % 10 != 0:
                        continue
                    for c in range(160):
                        if c % 10 != 0:
                            continue
                        feature.append(flows[r,c,0])
                        feature.append(flows[r,c,1])
                feature = np.array(feature)

                features_current_file.append(feature)

            prev_frame = frame
        print(category,'-----',filename)

        features.append({
            "filename": filename,
            "category": category,
            "features": features_current_file 
        })

        n_processed_files += 1
        if n_processed_files % 30 == 0:
            print("Done %d files" % n_processed_files)

    # Dump data to file.
    pickle.dump(features, open("data/optflow_%s.p" % category, "wb"))

Processing category boxing
boxing ----- person04_boxing_d4_uncomp.avi
boxing ----- person13_boxing_d2_uncomp.avi
boxing ----- person06_boxing_d2_uncomp.avi
boxing ----- person10_boxing_d2_uncomp.avi
boxing ----- person10_boxing_d1_uncomp.avi
boxing ----- person08_boxing_d3_uncomp.avi
boxing ----- person06_boxing_d1_uncomp.avi
boxing ----- person08_boxing_d4_uncomp.avi
boxing ----- person11_boxing_d3_uncomp.avi
boxing ----- person15_boxing_d4_uncomp.avi
boxing ----- person07_boxing_d4_uncomp.avi
boxing ----- person24_boxing_d3_uncomp.avi
boxing ----- person24_boxing_d1_uncomp.avi
boxing ----- person18_boxing_d1_uncomp.avi
boxing ----- person14_boxing_d3_uncomp.avi
boxing ----- person02_boxing_d4_uncomp.avi
boxing ----- person02_boxing_d1_uncomp.avi
boxing ----- person14_boxing_d2_uncomp.avi
boxing ----- person06_boxing_d3_uncomp.avi
boxing ----- person11_boxing_d1_uncomp.avi
boxing ----- person23_boxing_d4_uncomp.avi
boxing ----- person14_boxing_d1_uncomp.avi
boxing ----- person12_boxin

handclapping ----- person23_handclapping_d4_uncomp.avi
handclapping ----- person22_handclapping_d4_uncomp.avi
handclapping ----- person09_handclapping_d1_uncomp.avi
handclapping ----- person14_handclapping_d3_uncomp.avi
handclapping ----- person19_handclapping_d4_uncomp.avi
handclapping ----- person09_handclapping_d4_uncomp.avi
handclapping ----- person07_handclapping_d3_uncomp.avi
handclapping ----- person22_handclapping_d2_uncomp.avi
handclapping ----- person06_handclapping_d1_uncomp.avi
handclapping ----- person22_handclapping_d1_uncomp.avi
handclapping ----- person05_handclapping_d2_uncomp.avi
Done 180 files
handclapping ----- person16_handclapping_d1_uncomp.avi
handclapping ----- person05_handclapping_d3_uncomp.avi
handclapping ----- person15_handclapping_d1_uncomp.avi
handclapping ----- person20_handclapping_d4_uncomp.avi
handclapping ----- person23_handclapping_d1_uncomp.avi
handclapping ----- person13_handclapping_d4_uncomp.avi
handclapping ----- person16_handclapping_d4_uncomp

jogging ----- person21_jogging_d2_uncomp.avi
Done 330 files
jogging ----- person18_jogging_d4_uncomp.avi
jogging ----- person09_jogging_d4_uncomp.avi
jogging ----- person15_jogging_d4_uncomp.avi
jogging ----- person02_jogging_d4_uncomp.avi
jogging ----- person01_jogging_d2_uncomp.avi
jogging ----- person08_jogging_d3_uncomp.avi
jogging ----- person24_jogging_d2_uncomp.avi
jogging ----- person11_jogging_d4_uncomp.avi
jogging ----- person16_jogging_d4_uncomp.avi
jogging ----- person11_jogging_d1_uncomp.avi
jogging ----- person17_jogging_d1_uncomp.avi
jogging ----- person25_jogging_d3_uncomp.avi
jogging ----- person09_jogging_d3_uncomp.avi
jogging ----- person22_jogging_d4_uncomp.avi
jogging ----- person18_jogging_d1_uncomp.avi
jogging ----- person01_jogging_d4_uncomp.avi
jogging ----- person06_jogging_d1_uncomp.avi
jogging ----- person05_jogging_d1_uncomp.avi
jogging ----- person06_jogging_d3_uncomp.avi
jogging ----- person08_jogging_d4_uncomp.avi
jogging ----- person21_jogging_d1_uncomp

walking ----- person14_walking_d4_uncomp.avi
walking ----- person02_walking_d4_uncomp.avi
Done 510 files
walking ----- person10_walking_d4_uncomp.avi
walking ----- person13_walking_d3_uncomp.avi
walking ----- person18_walking_d1_uncomp.avi
walking ----- person07_walking_d4_uncomp.avi
walking ----- person12_walking_d1_uncomp.avi
walking ----- person23_walking_d2_uncomp.avi
walking ----- person19_walking_d1_uncomp.avi
walking ----- person06_walking_d4_uncomp.avi
walking ----- person14_walking_d2_uncomp.avi
walking ----- person01_walking_d2_uncomp.avi
walking ----- person08_walking_d3_uncomp.avi
walking ----- person09_walking_d4_uncomp.avi
walking ----- person21_walking_d4_uncomp.avi
walking ----- person13_walking_d4_uncomp.avi
walking ----- person02_walking_d1_uncomp.avi
walking ----- person02_walking_d2_uncomp.avi
walking ----- person11_walking_d4_uncomp.avi
walking ----- person04_walking_d1_uncomp.avi
walking ----- person11_walking_d2_uncomp.avi
walking ----- person04_walking_d3_uncomp

## 2. Make Dataset
Based on dataset splitting instruction on KTH page, split the computed optical flow with splitting instructions in KTH page and divide in train and test set. This also generates a files "train_keypoints.p" of all optical flow features in the train set whose format is specifically used in clustering.

In [20]:
# Dataset are divided according to the instruction at:
# http://www.nada.kth.se/cvap/actions/00sequences.txt
TRAIN_PEOPLE_ID = [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 1, 4]
TEST_PEOPLE_ID = [22, 2, 3, 5, 6, 7, 8, 9, 10]

train = []
test = []

# Store keypoints in training set.
train_keypoints = []

for category in CATEGORIES:
    print("Processing category %s" % category)
    category_features = pickle.load(open("data/optflow_%s.p" % category, "rb"))

    for video in category_features:
        person_id = int(video["filename"].split("_")[0][6:])

        if person_id in TRAIN_PEOPLE_ID:
            train.append(video)

            for frame in video["features"]:
                train_keypoints.append(frame)
        else:
            test.append(video)
        print(category,'-----',filename)

print("Saving train/test set to files")
pickle.dump(train, open("data/train.p", "wb"))
pickle.dump(test, open("data/test.p", "wb"))

print("Saving keypoints in training set")
pickle.dump(train_keypoints, open("data/train_keypoints.p", "wb"))

Processing category boxing
boxing ----- person09_walking_d2_uncomp.avi
boxing ----- person09_walking_d2_uncomp.avi
boxing ----- person09_walking_d2_uncomp.avi
boxing ----- person09_walking_d2_uncomp.avi
boxing ----- person09_walking_d2_uncomp.avi
boxing ----- person09_walking_d2_uncomp.avi
boxing ----- person09_walking_d2_uncomp.avi
boxing ----- person09_walking_d2_uncomp.avi
boxing ----- person09_walking_d2_uncomp.avi
boxing ----- person09_walking_d2_uncomp.avi
boxing ----- person09_walking_d2_uncomp.avi
boxing ----- person09_walking_d2_uncomp.avi
boxing ----- person09_walking_d2_uncomp.avi
boxing ----- person09_walking_d2_uncomp.avi
boxing ----- person09_walking_d2_uncomp.avi
boxing ----- person09_walking_d2_uncomp.avi
boxing ----- person09_walking_d2_uncomp.avi
boxing ----- person09_walking_d2_uncomp.avi
boxing ----- person09_walking_d2_uncomp.avi
boxing ----- person09_walking_d2_uncomp.avi
boxing ----- person09_walking_d2_uncomp.avi
boxing ----- person09_walking_d2_uncomp.avi
boxin

jogging ----- person09_walking_d2_uncomp.avi
jogging ----- person09_walking_d2_uncomp.avi
jogging ----- person09_walking_d2_uncomp.avi
jogging ----- person09_walking_d2_uncomp.avi
jogging ----- person09_walking_d2_uncomp.avi
jogging ----- person09_walking_d2_uncomp.avi
jogging ----- person09_walking_d2_uncomp.avi
jogging ----- person09_walking_d2_uncomp.avi
jogging ----- person09_walking_d2_uncomp.avi
jogging ----- person09_walking_d2_uncomp.avi
jogging ----- person09_walking_d2_uncomp.avi
jogging ----- person09_walking_d2_uncomp.avi
jogging ----- person09_walking_d2_uncomp.avi
jogging ----- person09_walking_d2_uncomp.avi
jogging ----- person09_walking_d2_uncomp.avi
jogging ----- person09_walking_d2_uncomp.avi
jogging ----- person09_walking_d2_uncomp.avi
jogging ----- person09_walking_d2_uncomp.avi
jogging ----- person09_walking_d2_uncomp.avi
jogging ----- person09_walking_d2_uncomp.avi
jogging ----- person09_walking_d2_uncomp.avi
jogging ----- person09_walking_d2_uncomp.avi
jogging --

walking ----- person09_walking_d2_uncomp.avi
walking ----- person09_walking_d2_uncomp.avi
walking ----- person09_walking_d2_uncomp.avi
walking ----- person09_walking_d2_uncomp.avi
walking ----- person09_walking_d2_uncomp.avi
walking ----- person09_walking_d2_uncomp.avi
walking ----- person09_walking_d2_uncomp.avi
walking ----- person09_walking_d2_uncomp.avi
walking ----- person09_walking_d2_uncomp.avi
walking ----- person09_walking_d2_uncomp.avi
walking ----- person09_walking_d2_uncomp.avi
walking ----- person09_walking_d2_uncomp.avi
walking ----- person09_walking_d2_uncomp.avi
walking ----- person09_walking_d2_uncomp.avi
walking ----- person09_walking_d2_uncomp.avi
walking ----- person09_walking_d2_uncomp.avi
walking ----- person09_walking_d2_uncomp.avi
walking ----- person09_walking_d2_uncomp.avi
walking ----- person09_walking_d2_uncomp.avi
walking ----- person09_walking_d2_uncomp.avi
walking ----- person09_walking_d2_uncomp.avi
walking ----- person09_walking_d2_uncomp.avi
walking --

## 3. Run K-means
Run K-means on "train_keypoints.p" with 200 as the number of clusters and produce the codebook.

In [21]:
dataset = "data/train_keypoints.p"
clusters = 200
    
print("Loading dataset")
train_features = pickle.load(open(dataset, "rb"))
n_features = len(train_features)

print("Number of feature points to run clustering on: %d" % n_features)

# Clustering with KMeans.
print("Running KMeans clustering")
kmeans = KMeans(init='k-means++',n_clusters=clusters,n_init=10,n_jobs=-1,verbose=1)
kmeans.fit(train_features)

# Save trained kmeans object to file.
pickle.dump(kmeans, open("data/cb_%dclusters.p" % clusters, "wb"))

Loading dataset
Number of feature points to run clustering on: 186805
Running KMeans clustering
Initialization complete
Initialization complete
Initialization complete
Initialization complete
Initialization complete
Initialization complete
Initialization complete
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 12666065.988341305
start iteration
done sorting
start iteration
done sorting
end inner loop
start iteration
done sorting
Iteration 0, inertia 12767692.536981942
start iteration
done sorting
end inner loop
end inner loop
start iteration
done sorting
end inner loop
Iteration 0, inertia 12726580.666625531
start iteration
done sorting
start iteration
done sorting
Iteration 1, inertia 12070681.566463403
start iteration
done sorting
end inner loop
start iteration
done sorting
Iteration 0, inertia 12751270.779842697
start iteration
end inner loop
done sorting
start iteration
done sorting
end inner loop
end inner loop
Iteration 0, inertia 12731748

done sorting
start iteration
done sorting
end inner loop
Iteration 11, inertia 11686758.926683325
start iteration
done sorting
end inner loop
end inner loop
end inner loop
end inner loop
end inner loop
end inner loop
Iteration 11, inertia 11700746.887960041
start iteration
done sorting
Iteration 12, inertia 11726060.375344615
start iteration
done sorting
end inner loop
Iteration 10, inertia 11693502.178226665
start iteration
done sorting
Iteration 12, inertia 11712758.408785254
start iteration
done sorting
Iteration 14, inertia 11689947.442641512
start iteration
done sorting
Iteration 11, inertia 11690524.44327298
start iteration
Iteration 11, inertia 11714355.501863284
done sorting
start iteration
done sorting
end inner loop
Iteration 12, inertia 11678750.40872251
start iteration
done sorting
end inner loop
end inner loop
end inner loop
end inner loop
end inner loop
Iteration 12, inertia 11693906.038507061
start iteration
done sorting
end inner loop
Iteration 13, inertia 11719730.5003

done sorting
end inner loop
end inner loop
Iteration 22, inertia 11642376.171257364
start iteration
done sorting
end inner loop
end inner loop
Iteration 23, inertia 11634846.427632872
start iteration
done sorting
end inner loop
Iteration 22, inertia 11652393.844534267
start iteration
done sorting
end inner loop
Iteration 24, inertia 11668522.733257776
start iteration
done sorting
Iteration 26, inertia 11657290.43928876
start iteration
done sorting
Iteration 24, inertia 11675707.44521225
start iteration
done sorting
end inner loop
end inner loop
Iteration 24, inertia 11654100.936164396
start iteration
done sorting
Iteration 23, inertia 11640534.605510004
start iteration
Iteration 23, inertia 11664359.180396399
done sorting
start iteration
done sorting
end inner loop
end inner loop
Iteration 24, inertia 11633887.297467958
start iteration
done sorting
end inner loop
end inner loop
end inner loop
end inner loop
Iteration 25, inertia 11666587.188931007
start iteration
done sorting
Iteration

start iteration
Iteration 38, inertia 11645968.096356379
done sorting
start iteration
done sorting
end inner loop
end inner loop
end inner loop
Iteration 35, inertia 11639618.0868029
end inner loop
start iteration
done sorting
Iteration 34, inertia 11641302.150908524
start iteration
done sorting
Iteration 36, inertia 11627872.534585677
start iteration
done sorting
end inner loop
end inner loop
Iteration 35, inertia 11628612.755598756
start iteration
done sorting
end inner loop
Iteration 36, inertia 11647365.702486115
start iteration
done sorting
Iteration 35, inertia 11652396.537115756
start iteration
done sorting
Iteration 36, inertia 11659380.62032258
start iteration
done sorting
Iteration 39, inertia 11645448.483916217
start iteration
done sorting
end inner loop
end inner loop
end inner loop
end inner loop
end inner loop
Iteration 35, inertia 11640504.201755198
start iteration
done sorting
Iteration 36, inertia 11638635.55249769
start iteration
done sorting
Iteration 37, inertia 116

start iteration
done sorting
end inner loop
end inner loop
Iteration 47, inertia 11639902.511356046
start iteration
done sorting
end inner loop
end inner loop
Iteration 47, inertia 11629884.187347014
start iteration
done sorting
Iteration 51, inertia 11641631.686774528
start iteration
done sorting
Iteration 47, inertia 11624849.36086155
start iteration
done sorting
Iteration 48, inertia 11653395.740815062
end inner loop
start iteration
end inner loop
done sorting
Iteration 46, inertia 11632281.966407254
start iteration
done sorting
end inner loop
end inner loop
Iteration 49, inertia 11625276.788188614
start iteration
done sorting
Iteration 47, inertia 11647835.563199755
start iteration
done sorting
end inner loop
end inner loop
Iteration 48, inertia 11639615.033664633
start iteration
done sorting
end inner loop
Iteration 48, inertia 11629614.44515526
Iteration 52, inertia 11641314.48842807
start iteration
start iteration
done sorting
done sorting
end inner loop
Iteration 49, inertia 11

...

Iteration 110, inertia 11644445.806553926
start iteration
done sorting
Iteration 110, inertia 11640785.868319215
start iteration
done sorting
end inner loop
end inner loop
end inner loop
end inner loop
end inner loop
Iteration 110, inertia 11619775.219654009
start iteration
done sorting
end inner loop
Iteration 114, inertia 11623776.06614571
start iteration
done sorting
Iteration 111, inertia 11640785.698031604
Iteration 109, inertia 11622552.90953483
start iteration
start iteration
done sorting
done sorting
Iteration 111, inertia 11635187.411407348
start iteration
done sorting
Iteration 111, inertia 11644438.042511513
start iteration
done sorting
end inner loop
end inner loop
end inner loop
end inner loop
end inner loop
Iteration 111, inertia 11619769.421081005
start iteration
done sorting
end inner loop
Iteration 115, inertia 11623750.348692548
start iteration
done sorting
Iteration 110, inertia 11622542.720349688
start iteration
done sorting
Iteration 112, inertia 11640785.613600083

start iteration
done sorting
end inner loop
Iteration 13, inertia 11687541.066608293
start iteration
done sorting
end inner loop
end inner loop
Iteration 146, inertia 11623368.947596582
start iteration
done sorting
Iteration 2, inertia 11940609.147226468
start iteration
done sorting
end inner loop
Iteration 14, inertia 11683572.068024902
start iteration
done sorting
end inner loop
Iteration 147, inertia 11623345.041438026
start iteration
done sorting
end inner loop
end inner loop
Iteration 15, inertia 11680359.187525686
start iteration
done sorting
Iteration 3, inertia 11869379.918831162
start iteration
done sorting
Iteration 148, inertia 11623310.482803216
start iteration
done sorting
end inner loop
end inner loop
end inner loop
Iteration 149, inertia 11623293.077192621
start iteration
done sorting
Iteration 16, inertia 11677470.612023858
start iteration
done sorting
end inner loop
Iteration 4, inertia 11826836.768173985
start iteration
done sorting
end inner loop
Iteration 150, inert

end inner loop
end inner loop
Iteration 183, inertia 11622004.276370779
start iteration
done sorting
Iteration 44, inertia 11623606.700157076
start iteration
done sorting
Iteration 30, inertia 11650555.517114565
start iteration
done sorting
end inner loop
end inner loop
end inner loop
Iteration 184, inertia 11621997.852988612
start iteration
done sorting
end inner loop
Iteration 45, inertia 11622219.408426201
start iteration
done sorting
Iteration 31, inertia 11649537.866983792
start iteration
done sorting
end inner loop
end inner loop
Iteration 185, inertia 11621993.554015635
start iteration
done sorting
end inner loop
Iteration 46, inertia 11620801.588362118
start iteration
done sorting
Iteration 32, inertia 11648620.508865701
start iteration
done sorting
end inner loop
end inner loop
Iteration 186, inertia 11621983.261034697
start iteration
done sorting
end inner loop
Iteration 47, inertia 11619741.300377859
start iteration
done sorting
Iteration 33, inertia 11647751.813342793
start

end inner loop
Iteration 62, inertia 11632583.732692575
start iteration
done sorting
Iteration 76, inertia 11611520.563489545
start iteration
done sorting
end inner loop
end inner loop
Iteration 63, inertia 11632535.569362786
start iteration
done sorting
Iteration 77, inertia 11611387.071285484
start iteration
done sorting
end inner loop
end inner loop
Iteration 64, inertia 11632485.609957766
start iteration
done sorting
Iteration 78, inertia 11611229.44756999
start iteration
done sorting
end inner loop
end inner loop
Iteration 65, inertia 11632437.444579462
start iteration
done sorting
Iteration 79, inertia 11611074.085442618
start iteration
done sorting
end inner loop
end inner loop
Iteration 66, inertia 11632372.911032846
start iteration
done sorting
Iteration 80, inertia 11610945.786126168
start iteration
done sorting
end inner loop
end inner loop
Iteration 67, inertia 11632334.438336903
start iteration
done sorting
Iteration 81, inertia 11610823.105212575
start iteration
done sort

Iteration 110, inertia 11631908.438024022
start iteration
done sorting
start iteration
done sorting
end inner loop
end inner loop
Iteration 125, inertia 11609041.37041677
start iteration
done sorting
Iteration 111, inertia 11631908.173350325
start iteration
done sorting
end inner loop
end inner loop
Iteration 126, inertia 11609032.98658627
start iteration
done sorting
Iteration 112, inertia 11631907.697796825
start iteration
done sorting
end inner loop
end inner loop
Iteration 127, inertia 11609022.88794125
start iteration
done sorting
Iteration 113, inertia 11631906.112497175
start iteration
done sorting
end inner loop
end inner loop
Iteration 128, inertia 11609009.715925697
start iteration
Iteration 114, inertia 11631904.822012976
start iteration
done sorting
done sorting
end inner loop
end inner loop
Iteration 129, inertia 11608994.954349967
Iteration 115, inertia 11631903.514388552
start iteration
done sorting
start iteration
done sorting
end inner loop
end inner loop
Iteration 130

## 4. Make Bag-Of-Words
Make BoW for every video in the training set, using the computed clusters with TF-IDF weighting scheme.
### 4.1 Define make_bow

In [22]:
def make_bow(dataset, clusters, tfidf):
    print("Make bow vector for each frame")

    n_videos = len(dataset)

    bow = np.zeros((n_videos, clusters.shape[0]), dtype=np.float)

    # Make bow vectors for all videos.
    video_index = 0
    for video in dataset:
        visual_word_ids = vq(video["features"], clusters)[0]
        for word_id in visual_word_ids:
            bow[video_index, word_id] += 1
        video_index += 1

    # Check whether to use TF-IDF weighting.
    if tfidf:
        print("Applying TF-IDF weighting")
        freq = np.sum((bow > 0) * 1, axis = 0)
        idf = np.log((n_videos + 1) / (freq + 1))
        bow = bow * idf

    # Replace features in dataset with the bow vector we've computed.
    video_index = 0
    for i in range(len(dataset)):

        dataset[i]["features"] = bow[video_index]
        video_index += 1

        if (i + 1) % 50 == 0:
            print("Processed %d/%d videos" % (i + 1, len(dataset)))

    return dataset

### 4.2 Make BoW for train

In [23]:
codebook_file = "data/cb_200clusters.p"
tfidf = 1
dataset = "data/train.p"
output = "data/train_bow_c200.p"

# Load clusters.
codebook = pickle.load(open(codebook_file, "rb"))
clusters = codebook.cluster_centers_

# Load dataset.
dataset = pickle.load(open(dataset, "rb"))

# Make bow vectors.
dataset_bow = make_bow(dataset, clusters, tfidf)

# Save.
pickle.dump(dataset_bow, open(output, "wb"))

Make bow vector for each frame
Applying TF-IDF weighting
Processed 50/383 videos
Processed 100/383 videos
Processed 150/383 videos
Processed 200/383 videos
Processed 250/383 videos
Processed 300/383 videos
Processed 350/383 videos


### 4.3 Make BoW for test

In [24]:
codebook_file = "data/cb_200clusters.p"
tfidf = 1
dataset = "data/test.p"
output = "data/test_bow_c200.p"

# Load clusters.
codebook = pickle.load(open(codebook_file, "rb"))
clusters = codebook.cluster_centers_

# Load dataset.
dataset = pickle.load(open(dataset, "rb"))

# Make bow vectors.
dataset_bow = make_bow(dataset, clusters, tfidf)

# Save.
pickle.dump(dataset_bow, open(output, "wb"))

Make bow vector for each frame
Applying TF-IDF weighting
Processed 50/216 videos
Processed 100/216 videos
Processed 150/216 videos
Processed 200/216 videos


## 5. Train SVM
Train linear SVM on BoW vectors of training set

In [25]:
def make_dataset(dataset):
    X = []
    Y = []

    for video in dataset:
        X.append(video["features"])
        Y.append(video["category"])

    return X, Y

dataset_bow = "data/train_bow_c200.p"
C = 1
output = "data/svm_C1_c200.p"
    
# Load and make dataset.
dataset = pickle.load(open(dataset_bow, "rb"))
X, Y = make_dataset(dataset)

# Train SVM and save to file.
clf = SVC(C=C,kernel="linear",verbose=True)
clf.fit(X, Y)
pickle.dump(clf, open(output, "wb"))

[LibSVM]

## 6. Evaluate SVM
Use computed SVM classifier to classify videos in train and test set and get the accuracy result

In [26]:
bow_file = "data/test_bow_c200.p"
svm_file = "data/svm_C1_c200.p"
data = pickle.load(open(bow_file, "rb"))

# Load trained SVM classifier.
clf = pickle.load(open(svm_file, "rb"))

confusion_matrix = np.zeros((6, 6))

correct = 0
for video in data:

    predicted = clf.predict([video["features"]])

    # Check if majority is correct.
    if predicted == video["category"]:
        correct += 1

    confusion_matrix[CATEGORIES.index(predicted),
        CATEGORIES.index(video["category"])] += 1

print("%d/%d Correct" % (correct, len(data)))
print("Accuracy =", correct / len(data))
    
print("Confusion matrix")
print(confusion_matrix)

171/216 Correct
Accuracy = 0.7916666666666666
Confusion matrix
[[32.  5.  3.  0.  0.  0.]
 [ 2. 31.  4.  0.  0.  1.]
 [ 0.  0. 29.  0.  0.  0.]
 [ 0.  0.  0. 22.  5.  7.]
 [ 1.  0.  0.  9. 29.  0.]
 [ 1.  0.  0.  5.  2. 28.]]
